In [24]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedKFold

In [17]:
books = pd.read_csv('input/transformed/books.csv', usecols=['book_idx', 'title_idx', 'author_idx', 'publisher_idx'])
users = pd.read_csv('input/transformed/users.csv', usecols=['user_idx', 'age', 'city_idx', 'province_idx', 'country_idx'])
train_ratings = pd.read_csv('input/transformed/train_ratings.csv', usecols=['user_idx', 'book_idx', 'rating'])
test_ratings = pd.read_csv('input/transformed/test_ratings.csv', usecols=['id', 'user_idx', 'book_idx'])

In [19]:
train = train_ratings.merge(users, on=['user_idx']).merge(books, on=['book_idx'])
train['rating'] -= 1

In [20]:

skf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
X, y = train.drop('rating', axis=1), train['axis']
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    lgb_params = {
        'objective': 'multiclass',
        'num_class': 10,
    }
    model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_eval)

    y_pred = model.predict(X_test)



,rating,user_idx,book_idx,age,city_idx,province_idx,country_idx,title_idx,author_idx,publisher_idx
0,5,114367,144740,33.0,1332,30,0,75170,9946,83
1,6,68490,144740,NaN,0,2,2,75170,9946,83
2,5,114367,12317,33.0,1332,30,0,9982,24,123
3,7,81814,12317,NaN,956,6,0,9982,24,123
4,10,170723,12317,43.0,427,0,0,9982,24,123
...,...,...,...,...,...,...,...,...,...,...
236660,8,222112,200644,NaN,55,0,0,73207,67012,904
236661,5,41752,83064,NaN,59,15,0,31193,12041,16
236662,5,167954,105103,35.0,8003,9,0,141314,2883,32
236663,10,27826,50677,22.0,2306,5,0,238544,1414,91
